In [51]:
import pandas as pd

In [52]:
migration_df = pd.read_csv('PEA03.20231022T221016.csv')
population_df = pd.read_csv('PEA11.20231013T111046.csv')

In [53]:
population_df.head()

,STATISTIC,STATISTIC Label,TLIST(A1),Year,C02076V03371,Single Year of Age,C02199V02655,Sex,UNIT,VALUE
0,PEA11,Population estimates from 1926,1926,1926,-,All ages,-,Both sexes,Number,2971992
1,PEA11,Population estimates from 1926,1926,1926,-,All ages,1,Male,Number,1506889
2,PEA11,Population estimates from 1926,1926,1926,-,All ages,2,Female,Number,1465103
3,PEA11,Population estimates from 1926,1926,1926,200,Under 1 year,-,Both sexes,Number,55391
4,PEA11,Population estimates from 1926,1926,1926,200,Under 1 year,1,Male,Number,28084


In [54]:
population_df = population_df.drop(['STATISTIC', 'TLIST(A1)', 'C02076V03371'], axis=1)
population_df

,STATISTIC Label,Year,Single Year of Age,C02199V02655,Sex,UNIT,VALUE
0,Population estimates from 1926,1926,All ages,-,Both sexes,Number,2971992
1,Population estimates from 1926,1926,All ages,1,Male,Number,1506889
2,Population estimates from 1926,1926,All ages,2,Female,Number,1465103
3,Population estimates from 1926,1926,Under 1 year,-,Both sexes,Number,55391
4,Population estimates from 1926,1926,Under 1 year,1,Male,Number,28084
...,...,...,...,...,...,...,...
11812,Population estimates from 1926,2023,98 years,1,Male,Number,197
11813,Population estimates from 1926,2023,98 years,2,Female,Number,632
11814,Population estimates from 1926,2023,99 years and over,-,Both sexes,Number,1600
11815,Population estimates from 1926,2023,99 years and over,1,Male,Number,352


In [55]:
populattion_df_age_groups_for_ml = population_df[(population_df['Single Year of Age'] != 'All ages') & (population_df['Year'] > 1995) & (population_df['Sex'] != 'Both sexes')]
populattion_df_age_groups_for_ml


,STATISTIC Label,Year,Single Year of Age,C02199V02655,Sex,UNIT,VALUE
3337,Population estimates from 1926,1996,Under 1 year,1,Male,Number,25231
3338,Population estimates from 1926,1996,Under 1 year,2,Female,Number,23623
3340,Population estimates from 1926,1996,1 year,1,Male,Number,24963
3341,Population estimates from 1926,1996,1 year,2,Female,Number,23611
3343,Population estimates from 1926,1996,2 years,1,Male,Number,25245
...,...,...,...,...,...,...,...
11810,Population estimates from 1926,2023,97 years,2,Female,Number,821
11812,Population estimates from 1926,2023,98 years,1,Male,Number,197
11813,Population estimates from 1926,2023,98 years,2,Female,Number,632
11815,Population estimates from 1926,2023,99 years and over,1,Male,Number,352


In [56]:
try:
    populattion_df_age_groups_for_ml['Single Year of Age'] = populattion_df_age_groups_for_ml['Single Year of Age'].replace('Under 1 year', '0 year')
    output = populattion_df_age_groups_for_ml.head()
except Exception as e:
    output = f"An error occurred: {e}"

output

C:\Users\mariy\AppData\Local\Temp\ipykernel_11284\2972797246.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  populattion_df_age_groups_for_ml['Single Year of Age'] = populattion_df_age_groups_for_ml['Single Year of Age'].replace('Under 1 year', '0 year')


,STATISTIC Label,Year,Single Year of Age,C02199V02655,Sex,UNIT,VALUE
3337,Population estimates from 1926,1996,0 year,1,Male,Number,25231
3338,Population estimates from 1926,1996,0 year,2,Female,Number,23623
3340,Population estimates from 1926,1996,1 year,1,Male,Number,24963
3341,Population estimates from 1926,1996,1 year,2,Female,Number,23611
3343,Population estimates from 1926,1996,2 years,1,Male,Number,25245


In [57]:
populattion_df_age_groups_for_ml['Single Year of Age'] = populattion_df_age_groups_for_ml['Single Year of Age'].str.extract(r'(\d+)').astype(int)
populattion_df_age_groups_for_ml


C:\Users\mariy\AppData\Local\Temp\ipykernel_11284\628222668.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  populattion_df_age_groups_for_ml['Single Year of Age'] = populattion_df_age_groups_for_ml['Single Year of Age'].str.extract(r'(\d+)').astype(int)


,STATISTIC Label,Year,Single Year of Age,C02199V02655,Sex,UNIT,VALUE
3337,Population estimates from 1926,1996,0,1,Male,Number,25231
3338,Population estimates from 1926,1996,0,2,Female,Number,23623
3340,Population estimates from 1926,1996,1,1,Male,Number,24963
3341,Population estimates from 1926,1996,1,2,Female,Number,23611
3343,Population estimates from 1926,1996,2,1,Male,Number,25245
...,...,...,...,...,...,...,...
11810,Population estimates from 1926,2023,97,2,Female,Number,821
11812,Population estimates from 1926,2023,98,1,Male,Number,197
11813,Population estimates from 1926,2023,98,2,Female,Number,632
11815,Population estimates from 1926,2023,99,1,Male,Number,352


In [58]:
def age_group(age):
    if age <= 14:
        return '0-14 years old'
    elif age <= 24:
        return '15-24 years old'
    elif age <= 44:
        return '25-44 years old'
    elif age <= 64:
        return '45-64 years old'
    else:
        return '65 and over'

populattion_df_age_groups_for_ml['Age Group'] = populattion_df_age_groups_for_ml['Single Year of Age'].apply(age_group)

populattion_df_age_groups_for_ml


C:\Users\mariy\AppData\Local\Temp\ipykernel_11284\2910215475.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  populattion_df_age_groups_for_ml['Age Group'] = populattion_df_age_groups_for_ml['Single Year of Age'].apply(age_group)


,STATISTIC Label,Year,Single Year of Age,C02199V02655,Sex,UNIT,VALUE,Age Group
3337,Population estimates from 1926,1996,0,1,Male,Number,25231,0-14 years old
3338,Population estimates from 1926,1996,0,2,Female,Number,23623,0-14 years old
3340,Population estimates from 1926,1996,1,1,Male,Number,24963,0-14 years old
3341,Population estimates from 1926,1996,1,2,Female,Number,23611,0-14 years old
3343,Population estimates from 1926,1996,2,1,Male,Number,25245,0-14 years old
...,...,...,...,...,...,...,...,...
11810,Population estimates from 1926,2023,97,2,Female,Number,821,65 and over
11812,Population estimates from 1926,2023,98,1,Male,Number,197,65 and over
11813,Population estimates from 1926,2023,98,2,Female,Number,632,65 and over
11815,Population estimates from 1926,2023,99,1,Male,Number,352,65 and over


In [59]:
migration_df.head(20)

,STATISTIC,STATISTIC Label,TLIST(A1),Year,C02076V03371,Age Group,C02199V02655,Sex,C02542V03077,Inward or Outward Flow,UNIT,VALUE
0,PEA03,Estimated Migration (Persons in April),1987,1987,-,All ages,-,Both sexes,1,Net migration,Thousand,-23.0
1,PEA03,Estimated Migration (Persons in April),1987,1987,-,All ages,-,Both sexes,4,Emigrants: All destinations,Thousand,40.2
2,PEA03,Estimated Migration (Persons in April),1987,1987,-,All ages,-,Both sexes,5,Immigrants: All origins,Thousand,17.2
3,PEA03,Estimated Migration (Persons in April),1987,1987,-,All ages,1,Male,1,Net migration,Thousand,NaN
4,PEA03,Estimated Migration (Persons in April),1987,1987,-,All ages,1,Male,4,Emigrants: All destinations,Thousand,NaN
5,PEA03,Estimated Migration (Persons in April),1987,1987,-,All ages,1,Male,5,Immigrants: All origins,Thousand,NaN
6,PEA03,Estimated Migration (Persons in April),1987,1987,-,All ages,2,Female,1,Net migration,Thousand,NaN
7,PEA03,Estimated Migration (Persons in April),1987,1987,-,All ages,2,Female,4,Emigrants: All destinations,Thousand,NaN
8,PEA03,Estimated Migration (Persons in April),1987,1987,-,All ages,2,Female,5,Immigrants: All origins,Thousand,NaN
9,PEA03,Estimated Migration (Persons in April),1987,1987,215,0 - 14 years,-,Both sexes,1,Net migration,Thousand,NaN


In [60]:
migration_df = migration_df.drop(['STATISTIC', 'TLIST(A1)', 'C02076V03371','C02199V02655','C02542V03077'  ], axis=1)
migration_df

,STATISTIC Label,Year,Age Group,Sex,Inward or Outward Flow,UNIT,VALUE
0,Estimated Migration (Persons in April),1987,All ages,Both sexes,Net migration,Thousand,-23.0
1,Estimated Migration (Persons in April),1987,All ages,Both sexes,Emigrants: All destinations,Thousand,40.2
2,Estimated Migration (Persons in April),1987,All ages,Both sexes,Immigrants: All origins,Thousand,17.2
3,Estimated Migration (Persons in April),1987,All ages,Male,Net migration,Thousand,NaN
4,Estimated Migration (Persons in April),1987,All ages,Male,Emigrants: All destinations,Thousand,NaN
...,...,...,...,...,...,...,...
1993,Estimated Migration (Persons in April),2023,65 years and over,Male,Emigrants: All destinations,Thousand,0.4
1994,Estimated Migration (Persons in April),2023,65 years and over,Male,Immigrants: All origins,Thousand,1.5
1995,Estimated Migration (Persons in April),2023,65 years and over,Female,Net migration,Thousand,1.8
1996,Estimated Migration (Persons in April),2023,65 years and over,Female,Emigrants: All destinations,Thousand,0.6


In [61]:
migration_df_age_group_for_ml = migration_df[(migration_df['Age Group'] != 'All ages') & (migration_df['Year'] > 1995) & (migration_df['Sex'] != 'Both sexes')]
migration_df_age_group_for_ml.head(10)

,STATISTIC Label,Year,Age Group,Sex,Inward or Outward Flow,UNIT,VALUE
498,Estimated Migration (Persons in April),1996,0 - 14 years,Male,Net migration,Thousand,NaN
499,Estimated Migration (Persons in April),1996,0 - 14 years,Male,Emigrants: All destinations,Thousand,0.4
500,Estimated Migration (Persons in April),1996,0 - 14 years,Male,Immigrants: All origins,Thousand,3.1
501,Estimated Migration (Persons in April),1996,0 - 14 years,Female,Net migration,Thousand,NaN
502,Estimated Migration (Persons in April),1996,0 - 14 years,Female,Emigrants: All destinations,Thousand,0.6
503,Estimated Migration (Persons in April),1996,0 - 14 years,Female,Immigrants: All origins,Thousand,3.6
507,Estimated Migration (Persons in April),1996,15 - 24 years,Male,Net migration,Thousand,NaN
508,Estimated Migration (Persons in April),1996,15 - 24 years,Male,Emigrants: All destinations,Thousand,9.8
509,Estimated Migration (Persons in April),1996,15 - 24 years,Male,Immigrants: All origins,Thousand,4.2
510,Estimated Migration (Persons in April),1996,15 - 24 years,Female,Net migration,Thousand,NaN


In [62]:
migration_df_age_group_for_ml.to_csv('net_migration.csv', index = False)

In [63]:
migration_df_age_group_for_ml.isna().sum()

STATISTIC Label            0
Year                       0
Age Group                  0
Sex                        0
Inward or Outward Flow     0
UNIT                       0
VALUE                     60
dtype: int64

We have some missing values for the net migration. However, the net migration is the difference between immigrants and emigrants, so this can be easily calculated and thos values can be imputated. 

In [64]:
pivot_table = migration_df_age_group_for_ml.pivot_table(
    values='VALUE', 
    index=['Year', 'Age Group', 'Sex'], 
    columns='Inward or Outward Flow', 
    aggfunc='first'
).reset_index()

# Calculate net migration if not already present
pivot_table['Net migration calculated'] = pivot_table['Immigrants: All origins'] - pivot_table['Emigrants: All destinations']

# Replace the existing net migration with the calculated one if it's NaN
pivot_table['Net migration'] = pivot_table['Net migration'].fillna(pivot_table['Net migration calculated'])

# Display the result
pivot_table[['Year', 'Age Group', 'Sex', 'Net migration']].head()

Inward or Outward Flow,Year,Age Group,Sex,Net migration
0,1996,0 - 14 years,Female,3.0
1,1996,0 - 14 years,Male,2.7
2,1996,15 - 24 years,Female,-5.0
3,1996,15 - 24 years,Male,-5.6
4,1996,25 - 44 years,Female,5.1


In [65]:
def calculate_net_migration(group):
    immigrants = group[group['Inward or Outward Flow'] == 'Immigrants: All origins']['VALUE'].values[0]
    emigrants = group[group['Inward or Outward Flow'] == 'Emigrants: All destinations']['VALUE'].values[0]
    net_migration = immigrants - emigrants
    group.loc[group['Inward or Outward Flow'] == 'Net migration', 'VALUE'] = net_migration
    return group

# Apply the function to each group
data_imputed = migration_df_age_group_for_ml.groupby(['Year', 'Age Group', 'Sex']).apply(calculate_net_migration).reset_index(drop=True)

# Display the updated dataframe to verify the imputation
data_imputed.head(20)

,STATISTIC Label,Year,Age Group,Sex,Inward or Outward Flow,UNIT,VALUE
0,Estimated Migration (Persons in April),1996,0 - 14 years,Female,Net migration,Thousand,3.0
1,Estimated Migration (Persons in April),1996,0 - 14 years,Female,Emigrants: All destinations,Thousand,0.6
2,Estimated Migration (Persons in April),1996,0 - 14 years,Female,Immigrants: All origins,Thousand,3.6
3,Estimated Migration (Persons in April),1996,0 - 14 years,Male,Net migration,Thousand,2.7
4,Estimated Migration (Persons in April),1996,0 - 14 years,Male,Emigrants: All destinations,Thousand,0.4
5,Estimated Migration (Persons in April),1996,0 - 14 years,Male,Immigrants: All origins,Thousand,3.1
6,Estimated Migration (Persons in April),1996,15 - 24 years,Female,Net migration,Thousand,-5.0
7,Estimated Migration (Persons in April),1996,15 - 24 years,Female,Emigrants: All destinations,Thousand,11.7
8,Estimated Migration (Persons in April),1996,15 - 24 years,Female,Immigrants: All origins,Thousand,6.7
9,Estimated Migration (Persons in April),1996,15 - 24 years,Male,Net migration,Thousand,-5.6
